# Test align

In [1]:
from koselleck import *

In [2]:
words=get_valid_words()#force=True,only_pos={'nn1'},max_rank=25000)#[:1000]
len(words)

6048

In [3]:
pathdf=get_pathdf_models()
pathdf

,corpus,period_start,period_end,path,path_vocab,run,period,period_len
403,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_01,1680-1690,10
410,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_02,1680-1690,10
392,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_03,1680-1690,10
401,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_04,1680-1690,10
405,bpo,1680,1690,/home/ryan/github/koselleck/data/models/bpo/16...,/home/ryan/github/koselleck/data/models/bpo/16...,run_05,1680-1690,10
...,...,...,...,...,...,...,...,...
747,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_21,1960-1970,10
756,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_22,1960-1970,10
754,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_23,1960-1970,10
739,bpo,1960,1970,/home/ryan/github/koselleck/data/models/bpo/19...,/home/ryan/github/koselleck/data/models/bpo/19...,run_24,1960-1970,10


In [4]:
m1=load_model_row(pathdf.query('period=="1700-1770"').iloc[0])

In [5]:
m2=load_model_row(pathdf.query('period=="1830-1900"').iloc[0])

In [6]:
#dfchange = measure_change(m1,m2,words=words,num_proc=1)

In [7]:
def measure_change_across_runs(
        pathdf = None,
        group_key='period',
        group1_name=None,
        group2_name=None,
        run_key='run',
        num_proc=1,
        words=None,
        num_runs=10,
        **attrs):
    if pathdf is None: pathdf=get_pathdf_models()
    pathdf = pathdf[~pathdf[group_key].isnull()]
    group_names=sorted(list(set(pathdf[group_key])))
    if len(group_names)<2: return pd.DataFrame()
    
    if not group1_name: group1_name=group_names[0]
    if not group2_name: group2_name=group_names[-1]
        
    pathdf_f = pathdf[pathdf[group_key].isin({group1_name,group2_name})]
    pathdf_f = pd.concat([
        grp
        for gi,grp in pathdf_f.groupby(run_key)
        if len(grp)==2
    ][:num_runs])
    
    dfruns = pmap_groups(
        do_measure_changes,
        pathdf_f.groupby(run_key),
        num_proc=num_proc,
        kwargs=dict(
            words=words,
            group_key=group_key,
            progress=1
        ),
        progress=1,
        desc=f'Measuring change across {pathdf_f[run_key].nunique()} period/run model pairs'
    )
    return dfruns

def do_measure_changes(rundf,group_key='period',words=None,**attrs):
    rundf=rundf.sort_values(group_key)
#     display(rundf)
    m1=load_model_row(rundf.iloc[0])
    m2=load_model_row(rundf.iloc[1])
    odf=measure_change(m1,m2,words=words,**attrs).reset_index()
    return odf

In [8]:
dfruns=measure_change_across_runs(
    pathdf,#[pathdf.run.isin({'run_01','run_02','run_03','run_04'})],
    words=words,
    num_proc=5,
    num_runs=10
)
dfruns

Measuring change in abstractness: 100%|██████████| 1353/1353 [00:00<00:00, 11219.71it/s]
Aligning noise aware matrices: iter: 13; alpha: 0.44; sigma: 0.065; sigmay: 1.318: : 14it [00:02,  6.12it/s] 
Aligning noise aware matrices: iter: 15; alpha: 0.438; sigma: 0.065; sigmay: 1.316: : 16it [00:02,  6.21it/s]
Aligning noise aware matrices: iter: 14; alpha: 0.441; sigma: 0.066; sigmay: 1.322: : 15it [00:02,  6.00it/s]
Aligning noise aware matrices: iter: 15; alpha: 0.432; sigma: 0.063; sigmay: 1.304: : 17it [00:03,  3.70it/s]
Calculating local neighborhood distances [x1]: 100%|██████████| 1353/1353 [00:21<00:00, 63.99it/s]12.18it/s] 
Aligning noise aware matrices: iter: 15; alpha: 0.437; sigma: 0.065; sigmay: 1.316: : 16it [00:01, 12.21it/s]
Calculating local neighborhood distances [x1]: 100%|██████████| 1353/1353 [00:21<00:00, 62.46it/s] 
Aligning noise aware matrices: iter: 10; alpha: 0.443; sigma: 0.067; sigmay: 1.322: : 12it [00:01,  8.31it/s]
Aligning noise aware matrices: iter: 14; 

,word,rank,perc,dist,neighborhood1_local,neighborhood2_local,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,...,dist_noiseaware,dist_procrustes,dist_local,score1_abstractness,score2_abstractness,contrast_abstractness,source_abstractness,period_abstractness,score_diff_abstractness,is_clean_noiseaware
run,,,,,,,,,,,,,,,,,,,,,
run_01,fort,1.75,99.944568,3.902355,"-sense, -consequence, -doubt, -faith, -experie...","+reign, +bay, +retreat, +castle, +monsieur, +f...",4.0,1.0,1.0,1.0,...,2.737795,2.783341,6.805855,2.739533,-0.298719,Abs-Conc,Median,orig,-3.038252,False
run_01,square,18.00,98.743533,2.794887,"-interpretation, -complaint, -envy, -propositi...","+street, +pic, oak, +tower, +temple, +toy, +la...",44.0,21.0,3.0,4.0,...,1.959880,2.707158,4.353085,1.372510,-0.932608,Abs-Conc,Median,orig,-2.305118,False
run_01,liberty,20.00,98.595713,3.663050,"-conscience, -law, -government, -religion, -st...","+bra, +lace, +bow, +brass, +cap, +hem, +gown, ...",1.0,67.0,9.0,3.0,...,1.627319,2.289991,4.648114,3.115316,-1.753704,Abs-Conc,Median,orig,-4.869020,False
run_01,item,46.25,96.655580,2.142638,"-journey, -inn, -price, -exchange, -gate, -bib...","+answer, +purpose, +consideration, +sign, +for...",31.0,20.0,110.0,24.0,...,1.969961,1.474294,2.691940,-1.904670,0.579934,Abs-Conc,Median,orig,2.484604,False
run_01,health,48.50,96.489283,1.898904,"-rest, -wound, -fob, -ration, -colt, -chop, -s...","+cruelty, +beauty, +diet, +animal, +medicine, ...",82.0,22.0,14.0,76.0,...,1.936948,2.204532,1.716216,-0.944656,1.085292,Abs-Conc,Median,orig,2.029948,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
run_10,pa,1294.50,4.397635,-1.456552,"-tot, -tie, -ex, pe, tit, -tic, -aid, -ad, -li...","gi, +ilk, +mu, +fob, +mil, +tar, +ale, +fir, a...",1306.0,1311.0,1250.0,1311.0,...,-1.657826,-1.424861,-1.455819,-1.384616,-1.443423,Abs-Conc,Median,orig,-0.058808,True
run_10,brow,1296.50,4.249815,-1.458621,"-bot, -oat, -gin, -bro, -ore, -star, -row, -ti...","+forehead, cone, +nose, +slate, +dot, +heel, +...",1315.0,1282.0,1309.0,1280.0,...,-1.497402,-1.672689,-1.365074,-1.297022,-1.245897,Abs-Conc,Median,orig,0.051125,True
run_10,tit,1310.00,3.252033,-1.564685,"-gi, -oil, -pe, -tilt, -tic, -lot, -ton, -lid,...","pa, +fob, +gum, ivy, +fir, +vine, +mu, +ox, li...",1301.0,1336.0,1308.0,1295.0,...,-1.909725,-1.671875,-1.394807,-1.258766,-1.196407,Abs-Conc,Median,orig,0.062359,True


In [16]:
dfruns.to_csv('data/data.measured_change.runs.v3.csv')

In [17]:
def avg_change_across_runs(dfruns,num_proc=1):
    return pmap_groups(
        do_avg_change_across_runs,
        dfruns.groupby('word'),
        num_proc=num_proc,
        desc='Averaging data across runs'
    ).sort_values('rank')


def do_avg_change_across_runs(dfgrp):
    dfgrp_avg=dfgrp.mean()
    
    # neighbs combine
    def count_neighbs(neighbs):
        return Counter([
            w[1:] if not w[0].isalpha() else w
            for neighb in neighbs
            for w in str(neighb).split(', ')
            if w
        ])
    def reformat_neighbs(neighbs1,neighbs2):
        c1=count_neighbs(neighbs1)
        c2=count_neighbs(neighbs2)
        def do(c1,c2,x='-'):
            return ', '.join([
                f"{(x if not w in c2 else '')}{w} ({c})"
                for w,c in c1.most_common()
                if c>1
            ])
        return do(c1,c2,'-'), do(c2,c1,'+')
    
    neighbs1=dfgrp.neighborhood1_local
    neighbs2=dfgrp.neighborhood2_local
    dfgrp_avg['neighborhood1_local'],dfgrp_avg['neighborhood2_local']=reformat_neighbs(neighbs1,neighbs2)
    dfgrp_avg['score_diff_t_abstractness'],dfgrp_avg['score_diff_p_abstractness']=ttest_ind(
        dfgrp.score1_abstractness,
        dfgrp.score2_abstractness,
    )
        
    return pd.DataFrame([dfgrp_avg])


In [18]:
dfruns=pd.read_csv('data/data.measured_change.runs.v3.csv').set_index('word')#.drop('Unnamed: 0',1)
dfruns_avg=avg_change_across_runs(dfruns,num_proc=7)

Averaging data across runs [x7]: 100%|██████████| 1353/1353 [00:01<00:00, 730.72it/s]


In [19]:
dfruns_avg

,rank,perc,dist,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,perc_abstractness,perc_noiseaware,perc_procrustes,...,dist_procrustes,dist_local,score1_abstractness,score2_abstractness,score_diff_abstractness,is_clean_noiseaware,neighborhood1_local,neighborhood2_local,score_diff_t_abstractness,score_diff_p_abstractness
word,,,,,,,,,,,,,,,,,,,,,
fort,2.750,99.870658,3.672397,4.7,2.3,2.4,1.6,99.726534,99.903917,99.896526,...,2.638680,6.051708,3.014088,-0.213082,-3.227170,0.0,"-sense (10), -consequence (10), -faith (10), -...","+reign (10), +bay (10), +retreat (10), +castle...",52.148759,4.274046e-21
liberty,16.975,98.819290,3.647262,1.0,47.3,16.5,3.1,100.000000,96.577975,98.854398,...,2.249280,4.807268,2.900633,-1.814380,-4.715013,0.0,"-conscience (10), -law (10), -government (10),...","+bra (10), +lace (10), +bow (10), +brass (10),...",109.530235,7.055830e-27
item,42.125,96.960458,2.085301,47.0,23.8,63.0,34.7,96.600148,98.314856,95.417591,...,1.781070,2.368067,-1.699081,0.665146,2.364227,0.0,"-journey (10), -gate (10), -bar (10), -opera (...","+answer (10), +purpose (10), +consideration (1...",-39.445959,6.236951e-19
square,51.275,96.284183,2.384120,138.2,44.5,13.9,8.5,89.859571,96.784922,99.046563,...,2.381804,3.793086,1.076948,-0.857489,-1.934437,0.0,"-tour (8), -stress (8), -vanity (8), -protest ...","+street (10), +pic (10), oak (10), +tower (10)...",15.446481,7.882819e-12
lot,68.900,94.981523,2.368896,204.6,60.0,5.6,5.4,84.951959,95.639320,99.660015,...,2.541620,4.092508,-0.741799,0.912930,1.654729,0.0,"-hut (10), -ire (10), -tit (10), -lid (10), -g...","bit (10), +thinking (10), +sort (10), +chance ...",-28.046716,2.629273e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
soul,1238.750,8.518108,-1.480505,1083.3,1346.4,1313.3,1212.0,20.007391,0.561715,3.008130,...,-1.794046,-1.134393,0.975258,0.656953,-0.318305,1.0,"-fortune (10), -elephant (10), -finger (10), -...","+wit (10), imagination (10), humour (10), +voi...",4.140336,6.145376e-04
kid,1240.250,8.407243,-1.339473,1179.1,1272.8,1288.3,1220.8,12.926829,6.001478,4.855876,...,-1.597822,-1.185744,-1.405758,-1.403700,0.002058,1.0,"cot (10), -ax (10), -sis (10), -ton (9), -pal ...","tint (10), +beard (10), +tan (10), +hat (10), ...",-0.030967,9.756364e-01
pa,1253.150,7.453806,-1.386232,1167.8,1313.8,1244.7,1286.3,13.762010,2.971175,8.078344,...,-1.417038,-1.388599,-1.512746,-1.344265,0.168481,1.0,"-tie (10), lilt (10), -ad (10), tit (10), -ex ...","gi (10), +mu (10), ilk (10), +mil (10), ion (1...",-2.996679,7.741110e-03


In [20]:
dfruns_avg.to_csv('data/data.measured_change.runs_avg.v3.csv')

In [21]:
dfruns_avg.query('rank<1000').sample(n=10).sort_values('rank_local')

,rank,perc,dist,rank_abstractness,rank_noiseaware,rank_procrustes,rank_local,perc_abstractness,perc_noiseaware,perc_procrustes,...,dist_procrustes,dist_local,score1_abstractness,score2_abstractness,score_diff_abstractness,is_clean_noiseaware,neighborhood1_local,neighborhood2_local,score_diff_t_abstractness,score_diff_p_abstractness
word,,,,,,,,,,,,,,,,,,,,,
trip,297.050,78.118995,1.010978,901.2,65.0,62.0,160.0,33.466371,95.269771,95.491500,...,1.754263,1.190637,0.171755,-0.357917,-0.529672,0.0,"-forehead (9), -drawing (9), -debt (8), -oak (...","+journey (10), +voyage (10), +flight (10), tou...",6.092073,9.338287e-06
rest,245.850,81.903178,1.132798,708.8,53.9,43.4,177.3,47.686622,96.090170,96.866223,...,1.889503,1.131426,-0.497529,0.273054,0.770583,0.0,"-fox (8), -raft (8), -election (7), -wing (7),...","+peace (10), +desire (10), +sleep (10), journe...",-13.146438,1.146067e-10
person,445.950,67.113821,0.476616,761.7,392.2,448.2,181.7,43.776792,71.086475,66.947524,...,0.488606,1.087152,1.362365,0.662626,-0.699739,0.0,"-majesty (10), -prince (10), -life (10), -desi...","+man (10), +sect (10), woman (10), +word (10),...",12.664485,2.109117e-10
scotch,430.475,68.257576,0.582641,1033.8,303.3,173.5,211.3,23.665928,77.657058,87.250554,...,1.222533,1.058809,-0.843288,-0.486419,0.356869,0.0,"-motto (9), -committee (8), -tier (8), -hole (...","+gin (10), +taste (10), +bottle (10), +coffee ...",-5.527179,3.010211e-05
artillery,925.625,31.661123,-0.606861,1132.5,951.8,1032.3,585.9,16.371027,29.726534,23.776792,...,-0.820595,0.071899,-0.755776,-0.933182,-0.177406,1.0,"-camp (10), -evening (10), -congregation (9), ...","+chaplain (10), +bishop (10), deputy (9), +mem...",1.729395,1.008498e-01
measure,508.100,62.520325,0.309940,566.3,401.4,352.9,711.8,58.218773,70.406504,73.991131,...,0.716559,-0.193190,1.248870,0.282450,-0.966420,0.0,"-figure (8), -minute (8), -disease (8), -propo...","+order (10), +self (10), +suit (10), +satisfac...",12.928114,1.507322e-10
lamp,602.725,55.526608,0.120544,261.2,663.8,760.2,725.7,80.768662,51.012565,43.887657,...,-0.234534,-0.215052,-0.602085,-2.137266,-1.535181,0.0,"-deputy (7), -mount (7), -wound (7), -cup (7),...","flame (10), +pin (10), +rim (10), brass (10), ...",16.128189,3.808001e-12
butter,733.625,45.851811,-0.195598,770.5,630.1,705.1,828.8,43.126386,53.503326,47.960089,...,-0.106376,-0.414513,-0.474307,-1.169039,-0.694732,0.0,"-sweat (10), -gold (9), -hem (9), -slate (9), ...","+juice (10), pint (10), +pinch (10), +tin (10)...",10.173679,6.847497e-09
score,794.725,41.335920,-0.256594,1088.4,308.1,783.8,998.6,19.630451,77.302291,42.143385,...,-0.272463,-0.696600,0.068640,0.308856,0.240216,0.0,"scene (7), -vein (7), -election (7), -pack (7)...","+penalty (10), +figure (10), shot (10), record...",-3.315536,3.847047e-03


In [22]:
dfruns_avg.loc['value']

rank                                                                   470.425
perc                                                                   65.3049
dist                                                                  0.459567
rank_abstractness                                                       1011.4
rank_noiseaware                                                          145.4
rank_procrustes                                                          385.1
rank_local                                                               339.8
perc_abstractness                                                      25.3215
perc_noiseaware                                                        89.3274
perc_procrustes                                                        71.6112
perc_local                                                             74.9593
dist_abstractness                                                    -0.782772
dist_noiseaware                                     